#Downloads

In [ ]:
# !mv output/ old_2/

In [ ]:
# !zip -r centroids.zip output/centroids/

In [ ]:
!pip install tslearn

#Imports

In [ ]:
from datetime import datetime, timedelta
from google.colab import drive
from tslearn.clustering import silhouette_score, TimeSeriesKMeans
from tslearn.metrics import dtw

import csv
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pytz
import seaborn as sns
import time

#Functions

In [ ]:
def data_filtering_1(source, index):
  print("2. points_df")
  points_df = pd.read_csv(dir + 'output/points.csv')
  points_df = points_df.drop('Unnamed: 0', 1)
  print(points_df)

  print("3." + index + "__ma__norm.csv")
  # df = pd.read_csv(dir + source + '/' + index + '__ma__norm.csv')  # **
  df = pd.read_csv(dir + index + '__ma__norm.csv')
  df = df.drop('Unnamed: 0', 1)
  print(df)

  print("4. Filtered df")
  df['date'] = pd.to_datetime(df['date'])
  df.set_index('date', inplace=True)
  df.sort_index(inplace=True)
  
  return df

In [ ]:
def data_filtering_2(df, rep_sample_size):
  print(df)

  print("5. Dropped NA df")
  df.dropna(inplace=True)
  print(df)

  print("6. arr")
  arr = df.T.values
  print(arr)
  print(arr.shape)

  print("7. score_arr")
  score_arr = arr[:rep_sample_size]
  print(score_arr)
  print(score_arr.shape)
  print()
  print()

  return score_arr

In [ ]:
def silhouette(ts, score_arr, clusters):
  print("Silhouette")
  print("----------")
  scores = []

  # datetime object containing current date and time
  now = datetime.now(pytz.timezone('Israel'))
  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
  print("Running start time =", dt_string)

  start = time.time()

  for c in clusters:
    print("*** # clusters = " + str(c))
    model = TimeSeriesKMeans(n_clusters=c, max_iter=10, tol=1e-04, metric='dtw', n_jobs=-1, random_state=0)
    model.fit(score_arr)  # model
    score = silhouette_score(score_arr, model.labels_, metric='dtw')  # score
    scores.append(score)
    print(f'scores = {scores}')
    print()

    del score

  end = time.time()

  # datetime object containing current date and time
  now = datetime.now(pytz.timezone('Israel'))
  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
  print("Running end time =", dt_string)
  print(f'Total time elapsed: {timedelta(seconds=end - start)}')

  plt.plot(list(map(str, clusters)), scores, 'bx-')
  plt.xlabel('# Clusters')
  plt.ylabel('Score')
  plt.title(ts + '\n\nSilhouette scores')
  plt.savefig(dir + ts + '__silhouette.jpg')
  plt.show()

  row = [ts]
  for s in scores:
    row.append(str(s))

  with open(dir + 'output/scores.csv', 'a') as f: 
    write = csv.writer(f) 
    write.writerow(row) 

  # n_clusters_selected = scores.index(max(scores)) + 3
  # print(n_clusters_selected)

  print()
  print()

  # return n_clusters_selected

In [ ]:
def get_optimal_cluster(df, title):
  average_scores = list(df.mean(0))

  plt.bar(list(map(str, clusters)), average_scores)
  plt.xlabel('# Clusters')
  plt.ylabel('Avg. Silhouette Score')
  plt.title('Avg. Silhouette Scores: ' + title)

  for i in range(len(clusters)):
    plt.text(i, average_scores[i], "{:.3f}".format(average_scores[i]), ha='center')

  # plt.show()
  # print()
  
  max_val = max(average_scores)
  max_ix = average_scores.index(max_val) + clusters[0]

  plt.savefig(dir + '/output/optimal_cluster/' + '_'.join(title.split()) + '.jpg', transparent=True, bbox_inches='tight')
  plt.show()
  print()

  return max_ix

In [ ]:
def time_series_clustering(arr, n_clusters_selected, points_df, ts):
  print("Time Series Clustering")
  print("----------------------")
  # datetime object containing current date and time
  now = datetime.now(pytz.timezone('Israel'))
  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
  print("Running start time =", dt_string)

  start = time.time()

  print("*** # clusters selected = " + str(n_clusters_selected))
  model_selected = TimeSeriesKMeans(n_clusters=n_clusters_selected, max_iter=10, tol=1e-04, metric="dtw", n_jobs=-1, random_state=0)
  model_selected.fit(arr)
      
  end = time.time()

  # datetime object containing current date and time
  now = datetime.now(pytz.timezone('Israel'))
  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
  print("Running end time =", dt_string)
  print(f'Total time elapsed: {timedelta(seconds=end - start)}')

  pred_clusters = model_selected.predict(arr)

  print("Save Centroid to file ...")
  # open a file, where I want to store the data
  f = open(dir + 'output/centroids/' + ts, 'wb')

  # dump information to that file: Centroid
  pickle.dump(model_selected.cluster_centers_, f)

  # close the file
  f.close()

  ts_clusters = ts + '__' + str(n_clusters_selected)
  print(ts_clusters)

  points_df[ts_clusters] = pred_clusters
  print(points_df)
  print()
  print()

  points_df.to_csv(dir + 'output/points.csv')

#Mount Drive **

In [ ]:
# drive.mount('/content/drive')

#Constants

In [ ]:
dir = "/content/"
# dir = "/content/drive/MyDrive/Thesis/"  # **

In [ ]:
num_points = 20000
rep_sample_size = 2145

In [ ]:
# clusters = [2, 3, 4, 5]
clusters = [3, 4, 5]

In [ ]:
sources_and_indices = {'CHIRPS': ['precipitation'], 
                       'MODIS': ['lst_c'], 
                       'S1': ['b_asc_vh'], 
                       'S2': ['ci', 'ndvi', 'ndwi']}

In [ ]:
years = [2017, 2018, 2019, 2020, 2021]

In [ ]:
summer_months = [5, 6, 7, 8, 9, 10]
winter_months_1 = [11, 12]
winter_months_2 = [1, 2, 3, 4]

#Code

In [ ]:
# Done only once
header = ['source__index__year_months___clusters']
for c in clusters:
  header.append(str(c))

with open(dir + 'output/scores.csv', 'w') as f: 
  write = csv.writer(f) 
  write.writerow(header) 

##Silhouette

###1.&nbsp; Seasons over the Years (May-Oct, Nov-Apr 2017-2021; 9 Periods)

####Summers (May-Oct; 5 Periods)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
    print()
    
    for i in range(len(years)):
      
      print("Data Filtering")
      print("--------------")
      print("1. ts")
      
      ts = source + '__' + index + '__' + str(years[i]) + "_{:02d}".format(summer_months[0]) + "_{:02d}".format(summer_months[-1])
      
      print(ts)
      df = data_filtering_1(source, index)
      
      df = df[df.index.year.isin([years[i]]) & df.index.month.isin(summer_months)]
      df.sort_index(inplace=True)
      
      score_arr = data_filtering_2(df, rep_sample_size)
      silhouette(ts, score_arr, clusters)

####Winters (Nov-Apr; 4 Periods)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
    
    for i in range(len(years) - 1):
      
      print("Data Filtering")
      print("--------------")
      print("1. ts")
      
      ts = source + '__' + index + '__' + str(years[i]) + "_{:02d}".format(winter_months_1[0]) + '__' + str(years[i + 1]) + "_{:02d}".format(winter_months_2[-1])
      
      print(ts)
      df = data_filtering_1(source, index)
      
      df = df[(df.index.year.isin([years[i]]) & df.index.month.isin(winter_months_1)) | 
              (df.index.year.isin([years[i + 1]]) & df.index.month.isin(winter_months_2))]
      df.sort_index(inplace=True)
      
      score_arr = data_filtering_2(df, rep_sample_size)
      silhouette(ts, score_arr, clusters)

###2.&nbsp; Years (May-Apr; 4 Years)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
    
    for i in range(len(years) - 1):
      
      print("Data Filtering")
      print("--------------")
      print("1. ts")
      
      ts = source + '__' + index + '__' + str(years[i]) + "_{:02d}".format(summer_months[0]) + '__' + str(years[i + 1]) + "_{:02d}".format(winter_months_2[-1])
      
      print(ts)
      df = data_filtering_1(source, index)
      
      df = df[(df.index.year.isin([years[i]]) & df.index.month.isin(summer_months)) | 
              (df.index.year.isin([years[i]]) & df.index.month.isin(winter_months_1)) | 
              (df.index.year.isin([years[i + 1]]) & df.index.month.isin(winter_months_2))]
      df.sort_index(inplace=True)
      
      score_arr = data_filtering_2(df, rep_sample_size)
      silhouette(ts, score_arr, clusters)

###3.&nbsp; Seasons (May-Oct, Nov-Apr; 2 Periods)

####Summer (May-Oct)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")



    print("Data Filtering")
    print("--------------")
    print("1. ts")
    
    ts = source + '__' + index + '__' + "{:02d}".format(summer_months[0]) + "_{:02d}".format(summer_months[-1])
    
    print(ts)
    df = data_filtering_1(source, index)
    
    df = df[df.index.month.isin(summer_months)]
    df.sort_index(inplace=True)
    
    score_arr = data_filtering_2(df, rep_sample_size)
    silhouette(ts, score_arr, clusters)

####Winter (Nov-Apr)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
    
    
      
    print("Data Filtering")
    print("--------------")
    print("1. ts")
    
    ts = source + '__' + index + '__' + "{:02d}".format(winter_months_1[0]) + "_{:02d}".format(winter_months_2[-1])
    
    print(ts)
    df = data_filtering_1(source, index)
    
    df = df[df.index.month.isin(winter_months_1) | 
            df.index.month.isin(winter_months_2)]
    df.sort_index(inplace=True)
    
    score_arr = data_filtering_2(df, rep_sample_size)
    silhouette(ts, score_arr, clusters)

###4.&nbsp; 4.5 Years (May 2017 - Oct 2021)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
          
    print("Data Filtering")
    print("--------------")
    print("1. ts")
    
    ts = source + '__' + index + '__' + str(years[0]) + "_{:02d}".format(summer_months[0]) + '__' + str(years[-1]) + "_{:02d}".format(summer_months[-1])
    
    print(ts)
    df = data_filtering_1(source, index)
    
    df.sort_index(inplace=True)
    

    
    score_arr = data_filtering_2(df, rep_sample_size)
    silhouette(ts, score_arr, clusters)

In [ ]:
df = pd.read_csv(dir + 'output/scores.csv')
df

In [ ]:
df.columns

In [ ]:
for ix in df.index:
  ts = df['source__index__year_months___clusters'][ix]
  scores = [df['3'][ix], df['4'][ix], df['5'][ix]]
  plt.plot(list(map(str, clusters)), scores, 'bx-')
  plt.xlabel('# Clusters')
  plt.ylabel('Score')
  plt.title(ts + '\n\nSilhouette scores')
  plt.savefig(dir + '/output/silhouette_scores/' + ts + '__silhouette.jpg', transparent=True, bbox_inches='tight')
  plt.show()
  print()

## Finding the Optimal Number of Clusters (Which Maximizing the Average Silhouette Score)

###Entire Scores' Dataset

In [ ]:
df = pd.read_csv(dir + 'output/scores.csv')
df

In [ ]:
get_optimal_cluster(df, 'Entire Dataset')

###4.5 Years' Scores

In [ ]:
df_years = df[df['source__index__year_months___clusters'].str.contains('2017_05__2021_10')]
df_years

In [ ]:
get_optimal_cluster(df_years, '4.5 Years (May 2017 - Oct 2021)')

###Seasons' Scores

In [ ]:
df_summers = df[df['source__index__year_months___clusters'].str.contains('__05_10')]
df_winters = df[df['source__index__year_months___clusters'].str.contains('__11_04')]
df_seasons = pd.concat([df_summers, df_winters], axis=0)
df_seasons

In [ ]:
get_optimal_cluster(df_seasons, '\nSeasons (Summers (May-Oct) + Winters (Nov-Apr); 2017-2021)')

###Each Year's Score

In [ ]:
df_year = df.set_index('source__index__year_months___clusters').filter(regex='20\d\d_05__20\d\d_04', axis=0)
df_year

In [ ]:
get_optimal_cluster(df_year, 'Each Year (May-Apr 2017-2021)')

###Each Season's Score

In [ ]:
df_summer = df.set_index('source__index__year_months___clusters').filter(regex='20\d\d_05_10', axis=0)
df_winter = df.set_index('source__index__year_months___clusters').filter(regex='20\d\d_11__20\d\d_04', axis=0)
df_season = pd.concat([df_summer, df_winter], axis=0)
df_season

In [ ]:
get_optimal_cluster(df_season, '\nEach Year\'s Season (Summer (May-Oct) + Winter (Nov-Apr); 2017-2021)')

In [ ]:
df_summer

In [ ]:
get_optimal_cluster(df_summer, '\nEach Year\'s Summer (May-Oct; 2017-2021)')

In [ ]:
df_winter

In [ ]:
get_optimal_cluster(df_winter, '\nEach Year\'s Winter (Nov-Apr; 2017-2021)')

##Time-Series Clustering (for 3 Clusters)

In [ ]:
n_clusters_selected = 3

In [ ]:
points_df = pd.read_csv(dir + 'output/points.csv')
points_df.drop(['Unnamed: 0'], axis=1, inplace=True)
print(points_df)

###1.&nbsp; Seasons over the Years (May-Oct, Nov-Apr 2017-2021; 9 Periods)

####Summers (May-Oct; 5 Periods)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
    print()
    
    for i in range(len(years)):
      
      print("Data Filtering")
      print("--------------")
      print("1. ts")
      
      ts = source + '__' + index + '__' + str(years[i]) + "_{:02d}".format(summer_months[0]) + "_{:02d}".format(summer_months[-1])
      
      print(ts)
      df = data_filtering_1(source, index)
      
      df = df[df.index.year.isin([years[i]]) & df.index.month.isin(summer_months)]
      df.sort_index(inplace=True)
      
      arr = data_filtering_2(df, num_points)
      time_series_clustering(arr, n_clusters_selected, points_df, ts)

####Winters (Nov-Apr; 4 Periods)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
    
    for i in range(len(years) - 1):
      
      print("Data Filtering")
      print("--------------")
      print("1. ts")
      
      ts = source + '__' + index + '__' + str(years[i]) + "_{:02d}".format(winter_months_1[0]) + '__' + str(years[i + 1]) + "_{:02d}".format(winter_months_2[-1])
      
      print(ts)
      df = data_filtering_1(source, index)
      
      df = df[(df.index.year.isin([years[i]]) & df.index.month.isin(winter_months_1)) | 
              (df.index.year.isin([years[i + 1]]) & df.index.month.isin(winter_months_2))]
      df.sort_index(inplace=True)
      
      arr = data_filtering_2(df, num_points)
      time_series_clustering(arr, n_clusters_selected, points_df, ts)

###2.&nbsp; Years (May-Apr; 4 Years)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
    
    for i in range(len(years) - 1):
      
      print("Data Filtering")
      print("--------------")
      print("1. ts")
      
      ts = source + '__' + index + '__' + str(years[i]) + "_{:02d}".format(summer_months[0]) + '__' + str(years[i + 1]) + "_{:02d}".format(winter_months_2[-1])
      
      print(ts)
      df = data_filtering_1(source, index)
      
      df = df[(df.index.year.isin([years[i]]) & df.index.month.isin(summer_months)) | 
              (df.index.year.isin([years[i]]) & df.index.month.isin(winter_months_1)) | 
              (df.index.year.isin([years[i + 1]]) & df.index.month.isin(winter_months_2))]
      df.sort_index(inplace=True)
      
      arr = data_filtering_2(df, num_points)
      time_series_clustering(arr, n_clusters_selected, points_df, ts)

###3.&nbsp; Seasons (May-Oct, Nov-Apr; 2 Periods)

####Summer (May-Oct)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")



    print("Data Filtering")
    print("--------------")
    print("1. ts")
    
    ts = source + '__' + index + '__' + "{:02d}".format(summer_months[0]) + "_{:02d}".format(summer_months[-1])
    
    print(ts)
    df = data_filtering_1(source, index)
    
    df = df[df.index.month.isin(summer_months)]
    df.sort_index(inplace=True)
    
    arr = data_filtering_2(df, num_points)
    time_series_clustering(arr, n_clusters_selected, points_df, ts)

####Winter (Nov-Apr)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
    
    
      
    print("Data Filtering")
    print("--------------")
    print("1. ts")
    
    ts = source + '__' + index + '__' + "{:02d}".format(winter_months_1[0]) + "_{:02d}".format(winter_months_2[-1])
    
    print(ts)
    df = data_filtering_1(source, index)
    
    df = df[df.index.month.isin(winter_months_1) | 
            df.index.month.isin(winter_months_2)]
    df.sort_index(inplace=True)
    
    arr = data_filtering_2(df, num_points)
    time_series_clustering(arr, n_clusters_selected, points_df, ts)

###4.&nbsp; 4.5 Years (May 2017 - Oct 2021)

In [ ]:
for source in sources_and_indices.keys():
  indices = sources_and_indices[source]
  for index in indices:
    print("----------")
    print(index)
    print("----------")
          
    print("Data Filtering")
    print("--------------")
    print("1. ts")
    
    ts = source + '__' + index + '__' + str(years[0]) + "_{:02d}".format(summer_months[0]) + '__' + str(years[-1]) + "_{:02d}".format(summer_months[-1])
    
    print(ts)
    df = data_filtering_1(source, index)

    df.sort_index(inplace=True)
    

    
    arr = data_filtering_2(df, num_points)
    time_series_clustering(arr, n_clusters_selected, points_df, ts)

#End